In [1]:
import pint.models as model
import pint.toa as toa
import pint.logging
import pint.fitter
from pint.modelutils import model_ecliptic_to_equatorial
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord, ICRS, Galactic
import uncertainties as unc
import table_utils as tu
import glob

import pint.derived_quantities as dq
#See num2tex (https://github.com/AndrewChap/num2tex)
# > pip install num2tex
from num2tex import num2tex

pint.logging.setup(level="ERROR")

1

In [2]:
psrs = [
    "J0032+6946",
    "J0141+6303",
    "J0214+5222",
    "J0415+6111",
    "J0636+5128",
    "J0957-0619",
    "J1239+3239",
    "J1327+3423",
    "J1434+7257",
    "J1505-2524",
    "J1530-2114",
    "J1816+4510",
    "J1913+3732",
    "J1929+6630",
    "J1930+6205",
    "J2104+2830",
    "J2115+6702",
    "J2145+2158",
    "J2210+5712",
    "J2326+6243",
    "J2354-2250"
]

In [27]:
# Auto-populate rotational/timing params table
pars = sorted(glob.glob(f"data/*_fiore+22.par"))

f0ve = {}
f1ve = {}

for par in pars:
    with open(par, 'r') as infile:
        f2 = tu.ufve(0.0,0.0)
        for l in infile.readlines():
            if l.startswith("PSRJ"):
                psr     = l.split()[1]
                psr_tex = psr.replace('-','$-$')
            elif l.startswith("F0"):
                F0      = float(l.split()[1])
                F0_err  = float(l.split()[3])
                f0      = tu.ufve(F0, F0_err)
                f0ve[psr] = [F0*u.Hz, F0_err*u.Hz]
            elif l.startswith("F1"):
                F1      = float(l.split()[1])
                F1_err  = float(l.split()[3])
                f1      = tu.ufve(F1, F1_err)
                f1ve[psr] = [F1*u.Hz*u.s**-1, F1_err*u.Hz*u.s**-1]
            elif l.startswith("F2"):
                F2      = float(l.split()[1])
                F2_err  = float(l.split()[3])
                f2      = tu.ufve(F2, F2_err)
            elif l.startswith("PEPOCH"):
                epoch   = l.split()[1]
            elif l.startswith("START"):
                start   = float(l.split()[1])
            elif l.startswith("FINISH"):
                finish  = float(l.split()[1])
            elif l.startswith("TRES"):
                rms_us  = l.split()[1]
            elif l.startswith("NTOA"):
                ntoa    = l.split()[1]
            elif l.startswith("CHI2R"):
                chi2r   = float(l.split()[1])
                efac    = round(np.sqrt(chi2r), 4)
            else:
                pass
        span = f"{int(start)}--{int(finish)}"
    # '\' is a special python character, but prefixing with '\' turns it into a normal character print('\\') -> '\'
    out_str = f"{psr_tex} & {f0} & {f1} & {epoch} & {span} & {rms_us} & {ntoa} & {efac} \\\\"
    print(out_str.replace(" \\times ","\\times"))

J0032+6946 & 27.171118951999(1) & -2.65006(3)\times10^{-15} & 57606 & 55169--58303 & 19.299 & 1471 & 1.0769 \\
J0141+6303 & 21.42232413061(4) & -7.65(2)\times10^{-16} & 57312 & 57072--57789 & 75.126 & 122 & 1.0266 \\
J0214+5222 & 40.691271130929(3) & -4.9002(6)\times10^{-16} & 56974 & 55353--58594 & 74.765 & 902 & 1.0168 \\
J0415+6111 & 2.27174929826(6) & -2.8(4)\times10^{-16} & 57234 & 57071--57397 & 609.378 & 38 & 1.0396 \\
J0636+5128 & 348.55922629557(2) & -4.192(6)\times10^{-16} & 57277 & 56027--57397 & 4.516 & 9424 & 1.2398 \\
J0957$-$0619 & 0.58014345894(2) & -5(1)\times10^{-17} & 57220 & 57071--57369 & 767.042 & 67 & 1.067 \\
J1239+3239 & 212.71644800103(2) & -1.752(5)\times10^{-16} & 57733 & 56054--59412 & 21.483 & 283 & 1.1311 \\
J1327+3423 & 24.0890076960409(6) & -7.504(2)\times10^{-17} & 58334 & 56052--59055 & 3.023 & 2256 & 1.2044 \\
J1434+7257 & 23.957175000920(1) & -3.1476(4)\times10^{-16} & 56731 & 55196--58266 & 31.371 & 351 & 1.1144 \\
J1505$-$2524 & 1.000750212121(9) 

In [15]:
# Auto-populate derived common properties table
rave = {}
decve = {}
dmve = {}

for psr in psrs:
    par_path = f"data/{psr}_fiore+22.par"
    psr_tex = psr.replace('-','$-$')
    
    f0,f0err = f0ve[psr]
    f1,f1err = f1ve[psr]
    
    with open(par_path, 'r') as infile:
        for l in infile.readlines():
            if l.startswith("RAJ"):
                ra_str = l.split()[1]
                ra_err = float(l.split()[3])
                rave[psr] = [ra_str,ra_err]
            elif l.startswith("DECJ"):
                dec_str = l.split()[1].strip("+")
                dec_err = float(l.split()[3])
                decve[psr] = [dec_str,dec_err]
            elif l.startswith("DM "):
                dm_val = float(l.split()[1])
                try:
                    dm_err = float(l.split()[3])
                    dm = tu.ufve(dm_val,dm_err)
                    dmve[psr] = [dm_val,dm_err]
                except IndexError:
                    dm = dm_val
                    dmve[psr] = [dm_val,None]
            else:
                pass
            
    # Calculate derived quantities with PINT
    p,perr,pd,pderr = dq.pferrs(f0,f0err,f1,f1err)
    p_tex = tu.ufve(p.value,perr.value)
    pd_tex = tu.ufve(pd.value,pderr.value)
    age = f"{num2tex(dq.pulsar_age(f0,f1).value):.1e}" # yr
    bsurf = f"{num2tex(dq.pulsar_B(f0,f1).value):.1e}" # G
    edot = f"{num2tex(dq.pulsar_edot(f0,f1).value):.1e}" # erg/s
    ra = tu.format_ra(eqcoord,ra_err)
    dec = tu.format_dec(eqcoord,dec_err)
    gcoord = eqcoord.transform_to(Galactic)
    dmdist_ne, dmdist_ymw = tu.get_dmdists(gcoord,dm_val)
    
    out_str = f"{psr_tex} & {p_tex} & {pd_tex} & {age} & {bsurf} & {edot} & {dmdist_ne:.1f} & {dmdist_ymw:.1f} \\\\"
    
    print(out_str.replace(" \\times ","\\times"))

J0032+6946 & 0.036803784259552(1) & 3.58955(4)\times10^{-18} & 1.6\times10^{8} & 1.2\times10^{10} & 2.8\times10^{33} & 2.5 & 1.9 \\
J0141+6303 & 0.04668027586097(8) & 1.666(4)\times10^{-18} & 4.4\times10^{8} & 8.9\times10^{9} & 6.5\times10^{32} & 43.2 & 25.0 \\
J0214+5222 & 0.024575295197399(2) & 2.9595(4)\times10^{-19} & 1.3\times10^{9} & 2.7\times10^{9} & 7.9\times10^{32} & 0.9 & 1.1 \\
J0415+6111 & 0.44018941736(1) & 5.5(8)\times10^{-17} & 1.3\times10^{8} & 1.6\times10^{11} & 2.5\times10^{31} & 2.3 & 1.8 \\
J0636+5128 & 0.0028689528910995(2) & 3.458(5)\times10^{-21} & 1.3\times10^{10} & 1.0\times10^{8} & 5.8\times10^{33} & 0.6 & 0.6 \\
J0957$-$0619 & 1.72371158304(5) & 1.5(3)\times10^{-16} & 1.8\times10^{8} & 5.1\times10^{11} & 1.2\times10^{30} & 1.1 & 1.2 \\
J1239+3239 & 0.0047010939182060(4) & 3.87(1)\times10^{-21} & 1.9\times10^{10} & 1.4\times10^{8} & 1.5\times10^{33} & 0.8 & 0.9 \\
J1327+3423 & 0.041512710387168(1) & 1.2932(3)\times10^{-19} & 5.1\times10^{9} & 2.3\times10^{9} &

In [16]:
# Auto-populate coordinates/DM table
for psr in psrs:    
    psr_tex = psr.replace('-','$-$')
    
    ra_str,ra_err = rave[psr]
    dec_str,dec_err = decve[psr]
    dm,dm_err = dmve[psr]
    
    eqcoord = SkyCoord(ra_str,dec_str,frame=ICRS,unit=(u.hourangle, u.deg))

    ra = tu.format_ra(eqcoord,ra_err)
    dec = tu.format_dec(eqcoord,dec_err)
    gcoord = eqcoord.transform_to(Galactic)
    gl = f"{gcoord.l.deg:.2f}"
    gb = f"{gcoord.b.deg:.2f}".replace('-','$-$')
    print(f"{psr_tex} & {ra} & {dec} & {dm} & {gl} & {gb} \\\\")

J0032+6946 & $00^{\rm h}\, 32^{\rm m}\, 41\, \fs2477(3)$ & $+69\arcdeg\, 46\arcmin\, 28\, \farcs047(2)$ & 79.9988411646788 & 121.30 & 6.96 \\
J0141+6303 & $01^{\rm h}\, 41^{\rm m}\, 45\, \fs761(1)$ & $+63\arcdeg\, 03\arcmin\, 49\, \farcs445(9)$ & 272.76173351552245 & 128.60 & 0.75 \\
J0214+5222 & $02^{\rm h}\, 14^{\rm m}\, 55\, \fs2741(2)$ & $+52\arcdeg\, 22\arcmin\, 40\, \farcs912(3)$ & 22.036871495147285 & 135.63 & $-$8.42 \\
J0415+6111 & $04^{\rm h}\, 15^{\rm m}\, 51\, \fs63(5)$ & $+61\arcdeg\, 11\arcmin\, 51\, \farcs8(3)$ & 70.77680394997722 & 145.15 & 7.49 \\
J0636+5128 & $06^{\rm h}\, 36^{\rm m}\, 04\, \fs84747(6)$ & $+51\arcdeg\, 28\arcmin\, 59\, \farcs959(1)$ & 11.105636 & 163.91 & 18.64 \\
J0957$-$0619 & $09^{\rm h}\, 57^{\rm m}\, 08\, \fs12(2)$ & $-06\arcdeg\, 19\arcmin\, -37\, \farcs5(9)$ & 27.271309801895722 & 244.83 & 36.20 \\
J1239+3239 & $12^{\rm h}\, 39^{\rm m}\, 27\, \fs3140(1)$ & $+32\arcdeg\, 39\arcmin\, 23\, \farcs379(2)$ & 16.859043107320627 & 147.36 & 83.89 \\
J13

In [20]:
# Auto-populate proper motion, intrinsic P-dot table
psr_names = ['J1327+3423','J1434+7257','J1816+4510']
for psr in psr_names:
    par_path = f"data/{psr}_fiore+22.par" 
    
    psr_tex = psr.replace('-','$-$')
    ra_str, ra_err = rave[psr]
    dm, dm_err = dmve[psr]
    with open(par_path, 'r') as infile:
        for l in infile.readlines():
            if l.startswith("PMRA"):
                pmra_val = float(l.split()[1])
                pmra_err = float(l.split()[3])
                pmra = f"{tu.ufve(pmra_val,pmra_err)}".replace("-","$-$")
            elif l.startswith("PMDEC"):
                pmdec_val = float(l.split()[1])
                pmdec_err = float(l.split()[3])
                pmdec = f"{tu.ufve(pmdec_val,pmdec_err)}".replace("-","$-$")
            else:
                pass

    eqcoord = SkyCoord(ra_str,dec_str,frame=ICRS,unit=(u.hourangle, u.deg))
    gcoord = eqcoord.transform_to(Galactic)
    dmdist_ne, dmdist_ymw = tu.get_dmdists(gcoord,dm)

    for ii,dmdist in enumerate([dmdist_ne,dmdist_ymw]):
        dd,de = (dmdist,dmdist*0.3)*u.kpc
        dist = tu.ufve(dd.value,de.value)
        pmtot,pmtoterr = tu.PMtot_err(pmra_val*u.mas/u.yr,pmra_err*u.mas/u.yr,pmdec_val*u.mas/u.yr,pmdec_err*u.mas/u.yr)
        vt,vterr = tu.Vtrans_err(pmtot,pmtoterr,dd,de)
        vt_str = tu.ufve(vt.value,vterr.value)

        f0,f0err = f0ve[psr]
        f1,f1err = f1ve[psr]
        p,perr,pd,pderr = dq.pferrs(f0,f0err,f1,f1err)
        pdshk = dq.shklovskii_factor(pmtot,dd)*p.decompose()
        pds = f"{pdshk*1e21:.2f}"
        pdgal = tu.pd_gal(p,gcoord,dd).decompose()
        pdg = f"{pdgal*1e21:.2f}".replace("-","$-$")

        # Am I doing the math here correctly?
        pdint = pd-(pdshk+pdgal)
        pdi = f"{pdint.value*1e21:.2f}"

        # Re-derive age, bsurf, edot with intrinsic P-dot
        f1 = -1.0*pdint/p**2
        age = f"{dq.pulsar_age(f0,f1).value*1e-9:.1f}" # Gyr
        bsurf = f"{dq.pulsar_B(f0,f1).value*1e-8:.1f}" # 10^8 G
        edot = f"{dq.pulsar_edot(f0,f1).value*1e-33:.1f}" # 10^33 erg/s

        if ii == 0:
            print(f"{psr_tex} & {pmra} & {pmdec} & {dist} & {vt_str} & {pdg} & {pds} & {pdi} & {bsurf} & {age} & {edot} \\\\")
        elif ii == 1:
            print(f" & & & {dist} & {vt_str} & {pdg} & {pds} & {pdi} & {bsurf} & {age} & {edot} \\\\")

J1327+3423 & $-$7.8(1) & 5.5(2) & 0.4(1) & 17(5) & $-$4.95 & 3.47 & 130.80 & 23.6 & 5.0 & 0.1 \\
 & & & 0.3(1) & 16(5) & $-$4.74 & 3.22 & 130.84 & 23.6 & 5.0 & 0.1 \\
J1434+7257 & $-$4.5(9) & $-$7.6(6) & 0.6(2) & 26(8) & $-$6.20 & 4.96 & 549.65 & 48.5 & 1.2 & 0.3 \\
 & & & 1.1(3) & 5(1) \times 10^{1} & $-$8.18 & 8.86 & 547.73 & 48.4 & 1.2 & 0.3 \\
J1816+4510 & 2.5(3) & $-$4.4(3) & 2.4(7) & 6(2) \times 10^{1} & $-$0.84 & 0.48 & 43.44 & 3.8 & 1.2 & 52.7 \\
 & & & 4(1) & 1.0(3) \times 10^{2} & $-$1.42 & 0.87 & 43.63 & 3.8 & 1.2 & 52.9 \\


In [34]:
# Auto-populate ELL1 binary table
binary_psrs_ell1 = ['J0636+5128','J1239+3239','J1816+4510']
for bp in binary_psrs_ell1:
    par_path = f"data/{bp}_fiore+22.par"
    
    psr_tex = bp.replace('-','$-$')
    with open(par_path, 'r') as infile:
        pbdot_tex = "--"
        for l in infile.readlines():
            if l.startswith("A1"):
                a1_val = float(l.split()[1])
                a1_err = float(l.split()[3])
                a1_tex = tu.ufve(a1_val,a1_err)
            elif l.startswith("PB "):
                pb_val = float(l.split()[1])
                pb_err = float(l.split()[3])
                pb_tex = tu.ufve(pb_val,pb_err)
            elif l.startswith("PBDOT"):
                pbdot_val = float(l.split()[1])
                pbdot_err = float(l.split()[3])
                pbdot_tex = tu.ufve(pbdot_val,pbdot_err)
            elif l.startswith("TASC"):
                tasc_val = float(l.split()[1])
                tasc_err = float(l.split()[3])
                tasc_tex = tu.ufve(tasc_val,tasc_err)
            elif l.startswith("EPS1"):
                eps1_val = float(l.split()[1])
                eps1_err = float(l.split()[3])
                eps1_tex = tu.ufve(eps1_val,eps1_err)
            elif l.startswith("EPS2"):
                eps2_val = float(l.split()[1])
                eps2_err = float(l.split()[3])
                eps2_tex = tu.ufve(eps2_val,eps2_err)
            else:
                pass
    
    # Derive mass function and Mc,min
    fm = dq.mass_funct(pb_val*u.day,a1_val*u.lightsecond)
    fm_tex = f"{num2tex(fm.value):.4e}" # Msun
    mc = dq.companion_mass(pb_val*u.day,a1_val*u.lightsecond, i=90.0*u.deg, mp=1.4 * u.solMass)
    mc_tex = f"{mc.value:.2f}" # Msun

    outstr = f"{psr_tex} & {pb_tex} & {pbdot_tex} & {a1_tex} & {tasc_tex} & {eps1_tex} & {eps2_tex} & {fm_tex} & {mc_tex} \\\\"
    print(outstr.replace(" \\times ","\\times").replace(" -"," $-$").replace("$-$-","--"))

J0636+5128 & 0.0665513415(1) & 1.8(2)\times10^{-12} & 0.00898563(9) & 57277.0161637(6) & 2(2)\times10^{-5} & $-$5(2)\times10^{-5} & 1.7588\times10^{-7} & 0.01 \\
J1239+3239 & 4.085401710(6) & -- & 2.371127(2) & 57730.0802277(9) & 5(2)\times10^{-6} & 0(2)\times10^{-6} & 8.5759\times10^{-4} & 0.13 \\
J1816+4510 & 0.36089348743(5) & -- & 0.5953996(7) & 56945.0913679(2) & $-$6(3)\times10^{-6} & $-$4(2)\times10^{-6} & 1.7400\times10^{-3} & 0.16 \\


In [35]:
# Auto-populate DD binary table
binary_psrs_dd = ['J0032+6946','J0214+5222']

for bp in binary_psrs_dd:
    par_path = f"data/{bp}_fiore+22.par"
    
    psr_tex = bp.replace('-','$-$')
    with open(par_path, 'r') as infile:
        for l in infile.readlines():
            if l.startswith("A1"):
                a1_val = float(l.split()[1])
                a1_err = float(l.split()[3])
                a1_tex = tu.ufve(a1_val,a1_err)
            elif l.startswith("PB "):
                pb_val = float(l.split()[1])
                pb_err = float(l.split()[3])
                pb_tex = tu.ufve(pb_val,pb_err)
            elif l.startswith("T0"):
                t0_val = float(l.split()[1])
                t0_err = float(l.split()[3])
                t0_tex = tu.ufve(t0_val,t0_err)
            elif l.startswith("ECC"):
                ecc_val = float(l.split()[1])
                ecc_err = float(l.split()[3])
                ecc_tex = tu.ufve(ecc_val,ecc_err)
            elif l.startswith("OM"):
                om_val = float(l.split()[1])
                om_err = float(l.split()[3])
                om_tex = tu.ufve(om_val,om_err)
            else:
                pass

    # Derive mass function and Mc,min
    fm = dq.mass_funct(pb_val*u.day,a1_val*u.lightsecond)
    fm_tex = f"{num2tex(fm.value):.4e}" # Msun
    mc = dq.companion_mass(pb_val*u.day,a1_val*u.lightsecond, i=90.0*u.deg, mp=1.4 * u.solMass)
    mc_tex = f"{mc.value:.2f}" # Msun
    
    outstr = f"{psr_tex} & {pb_tex} & {a1_tex} & {t0_tex} & {ecc_tex} & {om_tex} & {fm_tex} & {mc_tex} \\\\"
    print(outstr.replace(" \\times ","\\times").replace(" -"," $-$"))

J0032+6946 & 527.621324(2) & 178.674771(3) & 57142.972(2) & 0.00053182(3) & 147.525(2) & 2.2000\times10^{-2} & 0.42 \\
J0214+5222 & 512.039768(2) & 174.565749(4) & 56638.6454(8) & 0.00532802(6) & 210.5907(5) & 2.1785\times10^{-2} & 0.42 \\
